# Project objective
In this project, we build a convolutional neural network model for classification of images of hand-written digits in MNIST dataset. 

Information about the dataset, some technical details about the used machine learning method(s) and mathematical details of the quantifications approaches are provided in the code. 

# Packages we work with in this notebook
We are going to use the following libraries and packages:

* **numpy**: NumPy is the fundamental package for scientific computing with Python. (http://www.numpy.org/)
* **sklearn**: Scikit-learn is a machine learning library for Python programming language. (https://scikit-learn.org/stable/)
* **keras**: keras is a widely-used neural network framework in python. 

In [1]:
import numpy as np
import keras 

# Introduction to the dataset

**Name**: MNIST hand-writtent digits

**Summary**: Recognizing hand-written digits  

**number of features**: 28 pixels in rows and 28 pixels in columns 

**Number of data points (instances)**: 70,000 (60,000 for trainign and 10,000 for test set)

**dataset accessibility**: Dataset is available as part of keras (https://keras.io/api/datasets/)




## Importing the dataset and splitting the data training and testing sets
We can easily load the data from keras. We need to then split the data to train and test, if we do not have a separate dataset for validation and/or testing, to make sure about generalizability of the model we train. However, the splitting also has been already taken care of in keras. So we can load the data as trainig and test sets.

In [2]:
from keras.datasets import mnist
#download mnist data and split into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


## Making sure about the dataset characteristics (number of data points and features)

In [4]:
print('Shapes of the images: {}'.format(X_train[0].shape))

Shapes of the images: (28, 28)


## Data preparation
We need to prepare the dataset for machine learnign modeling. Here we prepare the data in 2 steps:

1) Reshaping input features (pixels of images) to the shape that we can later use in the modeling.

2) Converting the integer array of labels to one-hot encodings to be used in neural network modeling

In [5]:
#reshape data to fit model
# Interpretation of the numbers in the reshape function
# 1) number of data points (images)
# 2 and 3) number of pixels in rows and columns
# 4) 1 stands for greyscale
X_train = X_train.reshape(60000,28,28,1)
X_test = X_test.reshape(10000,28,28,1)


from keras.utils import to_categorical
#one-hot encode target column
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print('One hot vector of the first image in the training set: {}'.format(y_train[0]))

One hot vector of the first image in the training set: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


## Building the supervised learning model
We want to build a multi-class classification model as the output variable include multiple classes. 


Here we build a neural network model with 2 hidden layers. A neural network with 2 or more hidden layers are called deep neural network. So technical it is a deep learning code. As you can see the implementation of a deep learning model is not difficult. But knowing how to interpret it, how to fine-tune the model and avoid overfitting are the parts that need experience and more knowledge.


### Convolutional neural network


In [8]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
# building a neural network model
model = Sequential()

# Adding 1st hidden layer with 32 as the number of output filters in the convolution
# input_dim should be specified as the number of input features
# kernel_size is the size of the convolutional filters. Here we are using 3*3 filteres.
model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(28,28,1)))

# Adding 2nd hidden layer with 16 as the number of output filters in the convolution
model.add(Conv2D(16, kernel_size=3, activation='relu'))

# We want to flatten the image shape matrices of the last hidden layer
# and then use the results in the output layer
model.add(Flatten())

# adding the output layer (softmax is used to generate probabilities for each predicted class)
# Size if the last layer should be equal to the total number of classes in the dataset which is 10
model.add(Dense(10, activation='softmax'))

# compiling the model using cross-entropy for categorical variables,
# as we are dealing with multi-class classification
# Adam optimization algorithm is also used
# Accuracy is used as the metric to assess performance of our model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Now we fit our neural network model using the training set:

In [9]:
# Train the model using the training set
# We can also check the performance of the model after every epoch on the validation set. 
# Here we use the test set of MNIST dataset to check the performance on the validation set.
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)


Epoch 1/10
1875/1875 [==============================] - 67s 35ms/step - loss: 0.2691 - accuracy: 0.9467 - val_loss: 0.1042 - val_accuracy: 0.9680
Epoch 2/10
1875/1875 [==============================] - 66s 35ms/step - loss: 0.0754 - accuracy: 0.9775 - val_loss: 0.0922 - val_accuracy: 0.9738
Epoch 3/10
1875/1875 [==============================] - 66s 35ms/step - loss: 0.0522 - accuracy: 0.9840 - val_loss: 0.0887 - val_accuracy: 0.9735
Epoch 4/10
1875/1875 [==============================] - 66s 35ms/step - loss: 0.0390 - accuracy: 0.9875 - val_loss: 0.0848 - val_accuracy: 0.9780
Epoch 5/10
1875/1875 [==============================] - 66s 35ms/step - loss: 0.0307 - accuracy: 0.9902 - val_loss: 0.1162 - val_accuracy: 0.9758
Epoch 6/10
1875/1875 [==============================] - 67s 36ms/step - loss: 0.0283 - accuracy: 0.9913 - val_loss: 0.1233 - val_accuracy: 0.9775
Epoch 7/10
1875/1875 [==============================] - 67s 35ms/step - loss: 0.0207 - accuracy: 0.9938 - val_loss: 0.1485 -

***Note***. Accuracy of the model in the validation set is higher than training set after the first epoch. This is not something unusual. The model could perform better in teh validation set after the first few epochs as the number and distribution of datapoints are different between the trainign and validation sets. However, as the model gets better and better, trainign set accuracy goes abovevalidation set accuracy.

**Early stopping**: The concept of early stopping tells us that we can stop when the validation loss starts increasing in a consistent way, not small increases due to oscillatory behavior of the optimization process. Hence, stopping the model after 4 epochs of training for this model would have been a good choice.

The model is trained now and can be used to predict the lables of datapoints in the test set. To be able to assess the performance of the predictions in the test set using metrics class in sklearn, we need to transform the true lables and the predictions from one-hot encodings to lists.

In [10]:
y_pred = model.predict(X_test)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))

## Evaluating performance of the model
We need to assess performance of the model using the predictions of the test set. We use accuracy and balanced accuracy. Here are their definitions:

* **recall** in this context is also referred to as the true positive rate or sensitivity

How many relevant item are selected




$${\displaystyle {\text{recall}}={\frac {tp}{tp+fn}}\,} $$

 

* **specificity** true negative rate



$${\displaystyle {\text{true negative rate}}={\frac {tn}{tn+fp}}\,}$$

* **accuracy**: This measure gives you a sense of performance for all the classes together as follows:

$$ {\displaystyle {\text{accuracy}}={\frac {tp+tn}{tp+tn+fp+fn}}\,}$$


\begin{equation*} accuracy=\frac{number\:of\:correct\:predictions}{(total\:number\:of\:data\:points (samples))} \end{equation*}


* **balanced accuracy**: This measure gives you a sense of performance for all the classes together as follows:

$${\displaystyle {\text{balanced accuracy}}={\frac {recall+specificity
}{2}}\,}$$


In [11]:
from sklearn import metrics

print('Accuracy of the neural network model is:', metrics.accuracy_score(pred,test)*100)

print("Blanced accuracy of the neural network model is:", metrics.balanced_accuracy_score(pred, test))

Accuracy of the neural network model is: 97.74000000000001
Blanced accuracy of the neural network model is: 0.9775660072665276
